# Importing needed libraries

In [1]:
!pip install hazm

  Obtaining dependency information for hazm from https://files.pythonhosted.org/packages/91/8c/cc3d01c27681eb8223781ea162a23f9926647ce864eb601a19aee4bce0af/hazm-0.10.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 35.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 47.7 MB/s eta 0:00:00
  Obtaining dependency information for python-crfsuite<0.10.0,>=0.9.9 from https://files.pythonhosted.org/packages/38/1d/c475ba7d11e9735f00eb08e2f5315aa2e21c24cc85a0474c3fd425edef58/python_crfsuite-0.9.10-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.3 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into a

In [2]:
!pip install urlextract

  Obtaining dependency information for uritools from https://files.pythonhosted.org/packages/6b/ff/b16f225ceeb47f5d8899371ce446a8d6c1fe509a8882998b869f2a794c25/uritools-4.0.2-py3-none-any.whl.metadata


In [3]:
!pip install emojis

In [4]:
from __future__ import unicode_literals
from hazm import *
import tensorflow as tf
from keras.models import Sequential
import pandas as pd
from keras.layers import Dense
import numpy as np
import re
from urlextract import URLExtract
import emojis
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adadelta,Adam,RMSprop
from keras.utils import to_categorical
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.metrics import f1_score, accuracy_score


# Importing data

In [5]:
data = pd.read_csv('/kaggle/input/snapfood-reveiews/Snappfood Reviews.csv', on_bad_lines='skip', delimiter='\t')

# Preview of data

In [6]:
data.head()

,Unnamed: 0,comment,label,label_id
0,0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
1,1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
2,2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
3,3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
4,4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0


In [7]:
data['comment'][100]

'نان خیلی خمیر بود!!!!!'

In [8]:
data['comment'][7498]

'آش خوشمزه\u200cای بود و حلیم هم بد نبود'

# Data preprocessing

In [9]:
def _multiple_replace(mapping, text):
    pattern = "|".join(map(re.escape, mapping.keys()))
    return re.sub(pattern, lambda m: mapping[m.group()], str(text))

def convert_fa_numbers(input_str):
    mapping = {
        '۰': '0',
        '۱': '1',
        '۲': '2',
        '۳': '3',
        '۴': '4',
        '۵': '5',
        '۶': '6',
        '۷': '7',
        '۸': '8',
        '۹': '9',
        '.': '.',
    }
    return _multiple_replace(mapping, input_str)


def convert_ar_characters(input_str):
    """
    Converts Arabic chars to related Persian unicode char
    :param input_str: String contains Arabic chars
    :return: New str with converted arabic chars
    """
    mapping = {
        'ك': 'ک',
        'ى': 'ی',
        'ي': 'ی',
        'ئ':'ی',
        'إ':'ا',
        'أ':'ا',
        'ة':'ه',
        'ؤ':'و'
    }
    return _multiple_replace(mapping, input_str)


def preprocess(text):
    extractor = URLExtract()
    for url in extractor.gen_urls(text):
        text = text.replace(url,'')
    
    text = convert_fa_numbers(text)
    text = convert_ar_characters(text)
    
    SMILEYS = {
        ":(": "ناراحت",
        ":)": "خوشحال",
        ":‑(": "ناراحت",
        ":‑)": "خوشحال",
        ":‑D": "خوشحال",
        ":D": "خوشحال",
        ";‑)": "خوشحال",
        ";)": "خوشحال",
        ":|": "ناراحت",
        ":‑|": "ناراحت",
        ":‑/": "ناراحت",
        ":/": "ناراحت",
        ":‑@": "عصبانی",
        ":@": "عصبانی"
        # Add more mappings as needed
    }
    for smiley, replacement in SMILEYS.items():
        text = text.replace(smiley, f'<{replacement}>')
        
    emj = emojis.get(text)
    for i in emj:
        if i in text:
            text = text.replace(i,'')

    # regex to detect and replace all smilies in the text with <smiley>
    text = re.sub(r"(:\s?\)|:-\)|\(\s?:|\(-:|:\'\)|:\s?D|8-\)|:\s?\||;\s?\)|:-\*|:-\||:-\(|:\s?P|:-P|:-p|:-b|:-O|:-o|:-0|:-\@|:\$|:-\^|:-&|:-\*|:-\+|:-\~|:-\`|:-\>|:-\<|:-\}|:-\{|\[:\s?\]|\[:\s?\]|:\s?\]|:\s?\[|:\s?\}|:\s?\{)",'<smiley>',text)
    text = text.lower() # we lowercase here to prevent changes in the URLs and smilies
    text = text.strip()
    text = re.sub(r'[<>#.:()"\'!?؟،,@$%^&*_+\[\]/]', ' ', text)
    text = re.sub(r'[\s]{2,}', ' ', text)
    text = re.sub(r'(\w)\1{2,}', r'\1',text)
    if re.search(r'[\u0600-\u06FF]', text):
        return(text)
    else:
        return 'None'

In [10]:
tqdm.pandas()

In [11]:
data['Cleaned'] = data['comment'].progress_apply(preprocess)

100%|██████████| 56700/56700 [41:13<00:00, 22.93it/s]


In [12]:
data.head()

,Unnamed: 0,comment,label,label_id,Cleaned
0,0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح
1,1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0,قرار بود 1 ساعته برسه ولی نیم ساعت زودتر از مو...
2,2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1,قیمت این مدل اصلا با کیفیتش سازگاری نداره فقط ...
3,3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0,عالی بود همه چه درست و به اندازه و کیفیت خوب ا...
4,4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0,شیرینی وانیلی فقط یک مدل بود


In [13]:
data = data.drop('Unnamed: 0', axis=1)

In [14]:
data

,comment,label,label_id,Cleaned
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0,قرار بود 1 ساعته برسه ولی نیم ساعت زودتر از مو...
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1,قیمت این مدل اصلا با کیفیتش سازگاری نداره فقط ...
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0,عالی بود همه چه درست و به اندازه و کیفیت خوب ا...
4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0,شیرینی وانیلی فقط یک مدل بود
...,...,...,...,...
56695,یک تیکه کم فرستاده بودن و با تماس من در کمترین...,HAPPY,0,یک تیکه کم فرستاده بودن و با تماس من در کمترین...
56696,عالی بود همه چیز ممنونم پیک هم خیلی مرتب و به ...,HAPPY,0,عالی بود همه چیز ممنونم پیک هم خیلی مرتب و به ...
56697,مثل همیشه عالی، من چندمین باره سفارش میدم و هر...,HAPPY,0,مثل همیشه عالی من چندمین باره سفارش میدم و هرب...
56698,دلستر استوایی خواسته بودم اما لیمویی فرستادند,HAPPY,0,دلستر استوایی خواسته بودم اما لیمویی فرستادند


In [15]:
data = data.dropna()

In [16]:
data

,comment,label,label_id,Cleaned
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0,قرار بود 1 ساعته برسه ولی نیم ساعت زودتر از مو...
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1,قیمت این مدل اصلا با کیفیتش سازگاری نداره فقط ...
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0,عالی بود همه چه درست و به اندازه و کیفیت خوب ا...
4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0,شیرینی وانیلی فقط یک مدل بود
...,...,...,...,...
56695,یک تیکه کم فرستاده بودن و با تماس من در کمترین...,HAPPY,0,یک تیکه کم فرستاده بودن و با تماس من در کمترین...
56696,عالی بود همه چیز ممنونم پیک هم خیلی مرتب و به ...,HAPPY,0,عالی بود همه چیز ممنونم پیک هم خیلی مرتب و به ...
56697,مثل همیشه عالی، من چندمین باره سفارش میدم و هر...,HAPPY,0,مثل همیشه عالی من چندمین باره سفارش میدم و هرب...
56698,دلستر استوایی خواسته بودم اما لیمویی فرستادند,HAPPY,0,دلستر استوایی خواسته بودم اما لیمویی فرستادند


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56700 entries, 0 to 56699
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   comment   56700 non-null  object
 1   label     56700 non-null  object
 2   label_id  56700 non-null  int64 
 3   Cleaned   56700 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.7+ MB


In [20]:
stopwords = [
    "و", "در", "به", "از", "که", "این", "را", "با", "های", "برای", "آن", "تا", "بر", "ها", "اند", "ای", "می",
    "ما", "دیگر", "یک", "درباره", "نیز", "او", "شما",
    "باید", "اگر", "چه", "اما", "توسط", "چون", "حتی", "وقتی", "بنابراین", "پس", "البته", "ولی", "همچنین", "اگرچه",
    "یکدیگر", "همین", "همه", "هر", "ولی", "تاکنون", "بیشتر", "چه", "دیروقت", "اول", "اخیر",
]

data['Cleaned_sw_rmvd'] = data['Cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))


In [22]:
data

,comment,label,label_id,Cleaned,Cleaned_sw_rmvd
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,واقعا حیف وقت بنویسم سرویس دهیتون شده افتضاح
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0,قرار بود 1 ساعته برسه ولی نیم ساعت زودتر از مو...,قرار بود 1 ساعته برسه نیم ساعت زودتر موقع رسید...
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1,قیمت این مدل اصلا با کیفیتش سازگاری نداره فقط ...,قیمت مدل اصلا کیفیتش سازگاری نداره فقط ظاهر فر...
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0,عالی بود همه چه درست و به اندازه و کیفیت خوب ا...,عالی بود درست اندازه کیفیت خوب امیداورم همیشه ...
4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0,شیرینی وانیلی فقط یک مدل بود,شیرینی وانیلی فقط مدل بود
...,...,...,...,...,...
56695,یک تیکه کم فرستاده بودن و با تماس من در کمترین...,HAPPY,0,یک تیکه کم فرستاده بودن و با تماس من در کمترین...,تیکه کم فرستاده بودن تماس من کمترین زمان برام ...
56696,عالی بود همه چیز ممنونم پیک هم خیلی مرتب و به ...,HAPPY,0,عالی بود همه چیز ممنونم پیک هم خیلی مرتب و به ...,عالی بود چیز ممنونم پیک هم خیلی مرتب موقع آورد
56697,مثل همیشه عالی، من چندمین باره سفارش میدم و هر...,HAPPY,0,مثل همیشه عالی من چندمین باره سفارش میدم و هرب...,مثل همیشه عالی من چندمین باره سفارش میدم هربار...
56698,دلستر استوایی خواسته بودم اما لیمویی فرستادند,HAPPY,0,دلستر استوایی خواسته بودم اما لیمویی فرستادند,دلستر استوایی خواسته بودم لیمویی فرستادند


In [21]:
data.to_csv('snappfood_comments_preprocessed.csv')

In [18]:
# def text_normalizer(text):
#     stemmer = Stemmer()
#     text = stemmer.stem(text)

#     normalizer = Normalizer()
#     text = normalizer.normalize(text)

#     return text

In [19]:
# data['Cleaned_normalized'] = data['Cleaned'].progress_apply(lambda x: text_normalizer(x))